# Double Package

Code examples from [Think Complexity, 2nd edition](https://thinkcomplex.com).

Copyright 2016 Allen Downey, [MIT License](http://opensource.org/licenses/MIT)

This is an adaptation of Allen Downey's `Boids7.py` to the Jupyter Notebook.

Note that `vpython` will not run on Python 3.13.

Controls:
- Double-click: place carrot
- Shift-drag: pan
- Control-drag: rotate
- Alt(Option)-drag: zoom

In [1]:
""" Code example from Think Complexity, by Allen Downey.

Original code by Matt Aasted, modified by Allen Downey.

Based on Reynolds, "Flocks, Herds and Schools" and
Flake, "The Computational Beauty of Nature."

Copyright 2011 Allen B. Downey.
Distributed under the MIT License.
"""
try:
    from vpython import *
except:
    print("This program requires VPython 7, which you can read about")
    print("at http://vpython.org/.  If you are using Anaconda, you can")
    print("install VPython by running the following on the command line:")
    print("conda install -c vpython vpython")
    raise ImportError

import numpy as np
from time import time
import random

<IPython.core.display.Javascript object>

Note that the canvas will be placed immediately above this cell (i.e. you may need to scroll up after executing the cells below).

In [6]:
null_vector = vector(0,0,0)


def random_vector(a, b):
    """Create a vector with each element uniformly distributed in [a, b)."""
    coords = np.random.uniform(a, b, size=3)
    return vector(*coords)


def limit_vector(vect):
    """If the magnitude is greater than 1, set it to 1"""
    if vect.mag > 1:
        vect.mag = 1
    return vect

def draw_wireframe_box(center, size, color=color.black):
    s = size / 2
    corners = [vector(x,y,z) for x in (-s,s) for y in (-s,s) for z in (-s,s)]
    edges = [
        (0,1), (0,2), (0,4), (1,3), (1,5), (2,3), (2,6), (3,7),
        (4,5), (4,6), (5,7), (6,7)
    ]
    for i, j in edges:
        curve(pos=[center + corners[i], center + corners[j]], color=color)

def random_pos():
    return vector(*np.random.uniform(-size, size, size=3))

class Boid(cone):
    """A Boid is a VPython cone with a velocity and an axis."""

    def __init__(self, radius=0.03, length=0.1):
        corner = vector(-size, -size, -size)
        jitter = np.random.uniform(0, 0.1, size=3)  # Small offset so they’re not overlapping
        pos = corner + vector(*jitter)
        self.vel = random_vector(0, 1).norm()
        cone.__init__(self, pos=pos, radius=radius, length=length, color=color.blue)
        self.axis = length * self.vel

    def get_neighbors(self, boids, radius, angle):
        """Return a list of neighbors within a field of view.

        boids: list of boids
        radius: field of view radius
        angle: field of view angle in radians

        returns: list of Boid
        """
        neighbors = []
        for boid in boids:
            if boid is self:
                continue
            offset = boid.pos - self.pos

            # if not in range, skip it
            if offset.mag > radius:
                continue

            # if not within viewing angle, skip it
            diff = self.vel.diff_angle(offset)
            if abs(diff) > angle:
                continue

            # otherwise add it to the list
            neighbors.append(boid)

        return neighbors

    def center(self, boids, radius=1, angle=1):
        """Find the center of mass of other boids in range and
        return a vector pointing toward it."""
        neighbors = self.get_neighbors(boids, radius, angle)
        vecs = [boid.pos for boid in neighbors]
        return self.vector_toward_center(vecs)

    def vector_toward_center(self, vecs):
        """Vector from self to the mean of vecs.

        vecs: sequence of vector

        returns: Vector
        """
        if vecs:
            center = np.mean(vecs)
            toward = vector(center - self.pos)
            return limit_vector(toward)
        else:
            return null_vector

    def avoid(self, boids, carrot, obstacles=None, radius=0.3, angle=np.pi):
        """Find the center of mass of all objects in range and
        return a vector in the opposite direction, with magnitude
        proportional to the inverse of the distance (up to a limit)."""
        objects = boids + [carrot]
        if obstacles:
            objects.extend(obstacles)
        neighbors = self.get_neighbors(objects, radius, angle)
        vecs = [obj.pos for obj in neighbors]
        return -self.vector_toward_center(vecs)

    def align(self, boids, radius=0.5, angle=1):
        """Return the average heading of other boids in range.

        boids: list of Boids
        """
        neighbors = self.get_neighbors(boids, radius, angle)
        vecs = [boid.vel for boid in neighbors]
        return self.vector_toward_center(vecs)

    def love(self, carrot):
        """Returns a vector pointing toward the carrot."""
        toward = carrot.pos - self.pos
        return limit_vector(toward)

    def set_goal(self, boids, carrot, obstacles=None):
        """Sets the goal to be the weighted sum of the goal vectors."""

        # weights for various rules
        w_avoid = 10
        w_center = 3
        w_align = 1
        w_love = 10

        self.goal = (w_center * self.center(boids) +
                     w_avoid * self.avoid(boids, carrot, obstacles) +
                     w_align * self.align(boids) +
                     w_love * self.love(carrot))
        self.goal.mag = 1

    def move(self, mu=0.1, dt=0.1, bounds=3, obstacles=None):
        """Update the velocity, position and axis vectors.

        mu: how fast the boids can turn (maneuverability).
        dt: time step
        """
        
        self.vel = (1-mu) * self.vel + mu * self.goal
        self.vel.mag = 1
        self.pos += dt * self.vel
        self.axis = self.length * self.vel
        # Bounce off the walls (in x, y, z directions)
        for axis in ['x', 'y', 'z']:
            if abs(getattr(self.pos, axis)) > bounds:
                # Invert velocity on that axis
                setattr(self.vel, axis, -getattr(self.vel, axis))
                # Move the boid back inside the bounds
                setattr(self.pos, axis, np.sign(getattr(self.pos, axis)) * bounds)
        if obstacles:
            for obstacle in obstacles:
                obs_min = obstacle.pos - obstacle.size / 2
                obs_max = obstacle.pos + obstacle.size / 2
                inside = all(getattr(self.pos, axis) > getattr(obs_min, axis) and
                             getattr(self.pos, axis) < getattr(obs_max, axis)
                             for axis in ['x', 'y', 'z'])

                if inside:
                    escape_dirs = {}
                    for axis in ['x', 'y', 'z']:
                        dist_to_min = abs(getattr(self.pos, axis) - getattr(obs_min, axis))
                        dist_to_max = abs(getattr(obs_max, axis) - getattr(self.pos, axis))
                        escape_dirs[axis] = min(dist_to_min, dist_to_max)

                    exit_axis = min(escape_dirs, key=escape_dirs.get)
                    setattr(self.vel, exit_axis, -getattr(self.vel, exit_axis))
                    if getattr(self.pos, exit_axis) < getattr(obstacle.pos, exit_axis):
                        setattr(self.pos, exit_axis, getattr(obs_min, exit_axis) - 0.01)
                    else:
                        setattr(self.pos, exit_axis, getattr(obs_max, exit_axis) + 0.01)

class World(object):

    def __init__(self, n=10):
        """Create n Boids and one carrot.

        tracking: indicates whether the carrot follows the mouse
        """
        self.boids = [Boid() for i in range(n)]
        self.carrot1 = sphere(pos=random_pos(),
                      radius=0.1,
                      color=vector(1, 0, 0))  # Red

        self.carrot2 = sphere(pos=random_pos(),
                      radius=0.1,
                      color=vector(0, 1, 0))  # Green

        # Start with carrot1 as the active target
        self.current_carrot = self.carrot1
        self.goal_stage = 1
        
        # Create 3 random obstacles that don't overlap with carrots
        self.obstacles = []
        obstacle_count = 3
        obstacle_size = vector(2, 2, 2)
        
        # Try random positions until we get 3 good ones
        while len(self.obstacles) < obstacle_count:
            rand_pos = vector(
                random.uniform(-size + 1, size - 1),
                random.uniform(-size + 1, size - 1),
                random.uniform(-size + 1, size - 1)
            )

            # Make sure it's far enough from both carrots
            too_close = False
            for carrot in [self.carrot1, self.carrot2]:
                if mag(rand_pos - carrot.pos) < 2:
                    too_close = True
                    break
            
            # Check if too close to existing obstacles
            for obs in self.obstacles:
                if mag(rand_pos - obs.pos) < 2: # tweak this threshold if needed
                    too_close = True 
                    break
                    
            if not too_close:
                obstacle = box(pos=rand_pos, size=obstacle_size, color=color.gray(0.5), opacity=0.4)
                self.obstacles.append(obstacle)
        
        self.tracking = False
        draw_wireframe_box(center=vector(0,0,0), size=2*size)

    def step(self):
        """Compute one time step."""
        # move the boids
        for boid in self.boids:
            boid.set_goal(self.boids, self.current_carrot, self.obstacles)
            boid.move(bounds=size, obstacles=self.obstacles)

        # if we're tracking, move the carrot
        if self.tracking:
            self.carrot.pos = scene.mouse.pos

In [8]:
n = 20
size = 5
scene = canvas()
scene.background = color.white

world = World(n)
scene.center = vector(0, 0, 0)
scene.autoscale = False

def toggle_tracking(evt):
    """If we're currently tracking, turn it off, and vice versa.
    """
    world.tracking = not world.tracking

# when the user clicks, toggle tracking.
scene.bind('click', toggle_tracking)

# Simulation start
start_time = time()
first_leg_done = False
second_leg_done = False

while 1:
    rate(10)
    world.step()

    for boid in world.boids:
        if not first_leg_done and mag(boid.pos - world.carrot1.pos) < 0.2:
            t1 = time()
            print(f"Carrot 1 reached in {t1 - start_time:.2f} seconds.")
            world.current_carrot = world.carrot2
            first_leg_done = True
            break

        elif first_leg_done and not second_leg_done and mag(boid.pos - world.carrot2.pos) < 0.2:
            t2 = time()
            print(f"Carrot 2 reached in {t2 - start_time:.2f} seconds.")
            second_leg_done = True
            break

    if second_leg_done:
        break


<IPython.core.display.Javascript object>

Carrot 1 reached in 9.14 seconds.
Carrot 2 reached in 19.42 seconds.
